In [29]:
import pandas as pd

In [30]:
result = pd.read_csv("../1.데이터셋/비상장추가데이터.csv")

In [31]:
result["회계년도"] = pd.to_datetime(result["회계년도"])
result = result[result["회계년도"].dt.month == 12].reset_index(drop=True)
result["회계년도"] = result["회계년도"].dt.year

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_5338/1217803801.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result["회계년도"] = pd.to_datetime(result["회계년도"])


In [32]:
result["[U01A100000000]자산(*)(IFRS연결)(천원)"] = result[
    "[U01A100000000]자산(*)(IFRS연결)(천원)"
].fillna(result["[U01A100000000]자산(*)(IFRS)(천원)"])

result["[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"] = result[
    "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"
].fillna(result["[U01B100000000]매출액(수익)(*)(IFRS)(천원)"])

print(result.isnull().sum())

회사명                                           0
거래소코드                                         0
회계년도                                          0
[U01210000]매출액(영업수익)(*)(천원)                  40
[U01110000]자산(*)(천원)                         40
[U01140000]부채(*)(천원)                         40
[U01160000]자본(*)(천원)                         40
[U01B100000000]매출액(수익)(*)(IFRS)(천원)      233068
[U01A100000000]자산(*)(IFRS)(천원)           233068
[U01A800000000]부채(*)(IFRS)(천원)           233068
[U01A600000000]자본(*)(IFRS)(천원)           233068
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)    233067
[U01A100000000]자산(*)(IFRS연결)(천원)         233067
[U01A800000000]부채(*)(IFRS연결)(천원)         240321
[U01A600000000]자본(*)(IFRS연결)(천원)         240321
자기자본순이익률                                     48
dtype: int64


In [33]:
result["[U01A100000000]자산(*)(IFRS연결)(천원)"] = result[
    "[U01A100000000]자산(*)(IFRS연결)(천원)"
].fillna(result["[U01110000]자산(*)(천원)"])

result["[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"] = result[
    "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)"
].fillna(result["[U01210000]매출액(영업수익)(*)(천원)"])

print(result.isnull().sum())

회사명                                           0
거래소코드                                         0
회계년도                                          0
[U01210000]매출액(영업수익)(*)(천원)                  40
[U01110000]자산(*)(천원)                         40
[U01140000]부채(*)(천원)                         40
[U01160000]자본(*)(천원)                         40
[U01B100000000]매출액(수익)(*)(IFRS)(천원)      233068
[U01A100000000]자산(*)(IFRS)(천원)           233068
[U01A800000000]부채(*)(IFRS)(천원)           233068
[U01A600000000]자본(*)(IFRS)(천원)           233068
[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)        39
[U01A100000000]자산(*)(IFRS연결)(천원)             39
[U01A800000000]부채(*)(IFRS연결)(천원)         240321
[U01A600000000]자본(*)(IFRS연결)(천원)         240321
자기자본순이익률                                     48
dtype: int64


In [34]:
result = result[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "[U01B100000000]매출액(수익)(*)(IFRS연결)(천원)",
        "[U01A100000000]자산(*)(IFRS연결)(천원)",
        "자기자본순이익률",
    ]
]

In [35]:
result.columns = ["회사명", "거래소코드", "회계년도", "매출액", "총자산", "자기자본순이익률"]

In [36]:
grouped = result.groupby("거래소코드")

In [37]:
new_df = pd.DataFrame()

In [38]:
for code, group in grouped:
    # print(code, group)
    # group["CROE"] = group["자기자본순이익률"].diff(1)
    group["총자산변화량"] = group["총자산"].diff(1)
    group["총자산증가율"] = group["총자산변화량"] / group["총자산"].shift(1) * 100
    group["총자산회전율"] = group["매출액"] / group["총자산"]

    new_df = pd.concat([new_df, group], ignore_index=True)

In [39]:
new_df

,회사명,거래소코드,회계년도,매출액,총자산,자기자본순이익률,총자산변화량,총자산증가율,총자산회전율
0,글로웨이(주),10037,2012,120584267.0,270483122.0,7.35,NaN,NaN,0.445811
1,글로웨이(주),10037,2013,140956622.0,222480735.0,-21.44,-48002387.0,-17.746907,0.633568
2,글로웨이(주),10037,2014,117002075.0,190690076.0,56.03,-31790659.0,-14.289174,0.613572
3,글로웨이(주),10037,2015,79172648.0,84944703.0,0.00,-105745373.0,-55.454052,0.932049
4,글로웨이(주),10037,2016,595211.0,27073112.0,0.00,-57871591.0,-68.128546,0.021985
...,...,...,...,...,...,...,...,...,...
243075,대한엔지니어링(주),107190,2022,5901771.0,4632684.0,0.00,-12870293.0,-73.532023,1.273942
243076,아인스인터내셔널주식회사,107194,2021,451291.0,14843137.0,-12.47,NaN,NaN,0.030404
243077,아인스인터내셔널주식회사,107194,2022,88635.0,24606144.0,16.59,9763007.0,65.774553,0.003602
243078,지디앤산업개발주식회사,107198,2021,0.0,12472523.0,0.00,NaN,NaN,0.000000


In [40]:
new_df[["회사명", "거래소코드", "회계년도", "총자산증가율", "총자산회전율"]].to_csv(
    "../1.데이터셋/피처생성/비상장추가피처.csv"
)

#### 비상장 CROE 만들기

In [41]:
audit_croe = pd.read_csv("../1.데이터셋/비상장CROE.csv")
audit_croe

,회사명,거래소코드,회계년도,[U01160000]자본(*)(천원),[U01167200] *당기순이익(천원),[U01A600000000]자본(*)(IFRS)(천원),[U01B840000000]당기순이익(손실)(IFRS)(천원),[U01A600000000]자본(*)(IFRS연결)(천원),[U01B840000000]당기순이익(손실)(IFRS연결)(천원)
0,(유)에스케이씨에보닉페록사이드코리아,93338,2019/12,42177028.0,9126233.0,NaN,NaN,NaN,NaN
1,(유)에스케이씨에보닉페록사이드코리아,93338,2020/12,43538920.0,9525627.0,NaN,NaN,NaN,NaN
2,(유)에스케이씨에보닉페록사이드코리아,93338,2021/12,44866981.0,9981742.0,NaN,NaN,NaN,NaN
3,(유)에스케이씨에보닉페록사이드코리아,93338,2022/12,49930560.0,7513312.0,NaN,NaN,NaN,NaN
4,(유)연세,93779,2019/12,1860117.0,1854091.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
243657,주식회사이에스아이,57932,2016/12,2806652.0,11297.0,NaN,NaN,NaN,NaN
243658,주식회사이에스아이,57932,2017/12,1638666.0,-1128078.0,NaN,NaN,NaN,NaN
243659,주식회사이에스아이,57932,2018/12,490337.0,-898923.0,NaN,NaN,NaN,NaN
243660,주식회사이에스아이,57932,2019/12,-725701.0,-941423.0,NaN,NaN,NaN,NaN


In [42]:
audit_croe["회계년도"] = pd.to_datetime(audit_croe["회계년도"])
audit_croe["회계년도"] = audit_croe["회계년도"].dt.year

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_5338/3632768233.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  audit_croe["회계년도"] = pd.to_datetime(audit_croe["회계년도"])


In [43]:
audit_croe["[U01B840000000]당기순이익(손실)(IFRS연결)(천원)"] = audit_croe[
    "[U01B840000000]당기순이익(손실)(IFRS연결)(천원)"
].fillna(audit_croe["[U01B840000000]당기순이익(손실)(IFRS)(천원)"])
print(audit_croe.isnull().sum())

회사명                                          0
거래소코드                                        0
회계년도                                         0
[U01160000]자본(*)(천원)                        42
[U01167200]      *당기순이익(천원)                 42
[U01A600000000]자본(*)(IFRS)(천원)          233541
[U01B840000000]당기순이익(손실)(IFRS)(천원)      233541
[U01A600000000]자본(*)(IFRS연결)(천원)        240923
[U01B840000000]당기순이익(손실)(IFRS연결)(천원)    233540
dtype: int64


In [44]:
audit_croe["[U01B840000000]당기순이익(손실)(IFRS연결)(천원)"] = audit_croe[
    "[U01B840000000]당기순이익(손실)(IFRS연결)(천원)"
].fillna(audit_croe["[U01167200]      *당기순이익(천원)"])
print(audit_croe.isnull().sum())

회사명                                          0
거래소코드                                        0
회계년도                                         0
[U01160000]자본(*)(천원)                        42
[U01167200]      *당기순이익(천원)                 42
[U01A600000000]자본(*)(IFRS)(천원)          233541
[U01B840000000]당기순이익(손실)(IFRS)(천원)      233541
[U01A600000000]자본(*)(IFRS연결)(천원)        240923
[U01B840000000]당기순이익(손실)(IFRS연결)(천원)        41
dtype: int64


In [45]:
audit_croe["[U01A600000000]자본(*)(IFRS연결)(천원)"] = audit_croe[
    "[U01A600000000]자본(*)(IFRS연결)(천원)"
].fillna(audit_croe["[U01A600000000]자본(*)(IFRS)(천원)"])
print(audit_croe.isnull().sum())

회사명                                          0
거래소코드                                        0
회계년도                                         0
[U01160000]자본(*)(천원)                        42
[U01167200]      *당기순이익(천원)                 42
[U01A600000000]자본(*)(IFRS)(천원)          233541
[U01B840000000]당기순이익(손실)(IFRS)(천원)      233541
[U01A600000000]자본(*)(IFRS연결)(천원)        233540
[U01B840000000]당기순이익(손실)(IFRS연결)(천원)        41
dtype: int64


In [46]:
audit_croe["[U01A600000000]자본(*)(IFRS연결)(천원)"] = audit_croe[
    "[U01A600000000]자본(*)(IFRS연결)(천원)"
].fillna(audit_croe["[U01160000]자본(*)(천원)"])
print(audit_croe.isnull().sum())

회사명                                          0
거래소코드                                        0
회계년도                                         0
[U01160000]자본(*)(천원)                        42
[U01167200]      *당기순이익(천원)                 42
[U01A600000000]자본(*)(IFRS)(천원)          233541
[U01B840000000]당기순이익(손실)(IFRS)(천원)      233541
[U01A600000000]자본(*)(IFRS연결)(천원)            41
[U01B840000000]당기순이익(손실)(IFRS연결)(천원)        41
dtype: int64


In [47]:
unlisted_croe = audit_croe[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "[U01A600000000]자본(*)(IFRS연결)(천원)",
        "[U01B840000000]당기순이익(손실)(IFRS연결)(천원)",
    ]
]

In [48]:
unlisted_croe.columns = ["회사명", "거래소코드", "회계년도", "자본", "당기순이익"]

In [49]:
unlisted_croe

,회사명,거래소코드,회계년도,자본,당기순이익
0,(유)에스케이씨에보닉페록사이드코리아,93338,2019,42177028.0,9126233.0
1,(유)에스케이씨에보닉페록사이드코리아,93338,2020,43538920.0,9525627.0
2,(유)에스케이씨에보닉페록사이드코리아,93338,2021,44866981.0,9981742.0
3,(유)에스케이씨에보닉페록사이드코리아,93338,2022,49930560.0,7513312.0
4,(유)연세,93779,2019,1860117.0,1854091.0
...,...,...,...,...,...
243657,주식회사이에스아이,57932,2016,2806652.0,11297.0
243658,주식회사이에스아이,57932,2017,1638666.0,-1128078.0
243659,주식회사이에스아이,57932,2018,490337.0,-898923.0
243660,주식회사이에스아이,57932,2019,-725701.0,-941423.0


In [50]:
unlisted_croe = unlisted_croe.fillna(0)

In [51]:
unlisted_croe["CROE"] = None

In [52]:
grouped = unlisted_croe.groupby("거래소코드")

In [53]:
for code, data in grouped:
    data_len = len(data)
    for i in range(data_len - 2):
        avg_asset_t = (
            data["자본"][data.index[i + 2]] + data["자본"][data.index[i + 1]]
        ) / 2
        avg_asset_t_1 = (data["자본"][data.index[i + 1]] + data["자본"][data.index[i]]) / 2
        soon_t = data["당기순이익"][data.index[i + 2]]
        soon_t_1 = data["당기순이익"][data.index[i + 1]]
        croe = (soon_t / avg_asset_t) - (soon_t_1 / avg_asset_t_1)
        unlisted_croe.loc[data.index[i + 2], "CROE"] = croe

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_5338/116943088.py:10: RuntimeWarning: invalid value encountered in scalar divide
  croe = (soon_t / avg_asset_t) - (soon_t_1 / avg_asset_t_1)


In [54]:
new_df = pd.merge(new_df, unlisted_croe, how="left", on=["회사명", "거래소코드", "회계년도"])
new_df

,회사명,거래소코드,회계년도,매출액,총자산,자기자본순이익률,총자산변화량,총자산증가율,총자산회전율,자본,당기순이익,CROE
0,글로웨이(주),10037,2012,120584267.0,270483122.0,7.35,NaN,NaN,0.445811,10039088.0,5469579.0,None
1,글로웨이(주),10037,2013,140956622.0,222480735.0,-21.44,-48002387.0,-17.746907,0.633568,6968850.0,-1823023.0,None
2,글로웨이(주),10037,2014,117002075.0,190690076.0,56.03,-31790659.0,-14.289174,0.613572,9708868.0,4671997.0,0.774641
3,글로웨이(주),10037,2015,79172648.0,84944703.0,0.00,-105745373.0,-55.454052,0.932049,-118249044.0,-115324627.0,1.564745
4,글로웨이(주),10037,2016,595211.0,27073112.0,0.00,-57871591.0,-68.128546,0.021985,-186135592.0,-67885317.0,-1.678963
...,...,...,...,...,...,...,...,...,...,...,...,...
243075,대한엔지니어링(주),107190,2022,5901771.0,4632684.0,0.00,-12870293.0,-73.532023,1.273942,-8894571.0,-16079831.0,None
243076,아인스인터내셔널주식회사,107194,2021,451291.0,14843137.0,-12.47,NaN,NaN,0.030404,55403.0,-3455.0,None
243077,아인스인터내셔널주식회사,107194,2022,88635.0,24606144.0,16.59,9763007.0,65.774553,0.003602,65424.0,10021.0,None
243078,지디앤산업개발주식회사,107198,2021,0.0,12472523.0,0.00,NaN,NaN,0.000000,-2427477.0,-2428477.0,None


In [55]:
new_df = new_df.fillna(0)

In [56]:
new_df.isna().sum()

회사명         0
거래소코드       0
회계년도        0
매출액         0
총자산         0
자기자본순이익률    0
총자산변화량      0
총자산증가율      0
총자산회전율      0
자본          0
당기순이익       0
CROE        0
dtype: int64

In [57]:
new_df[["회사명", "거래소코드", "회계년도", "총자산증가율", "총자산회전율", "CROE"]].to_csv(
    "../1.데이터셋/피처생성/비상장추가피처.csv"
)